<div>
<h3>Suggestion Mining</h3>


In [39]:
# import required packages

import pandas as pd 
import numpy as np
import os, gc, time, warnings


from scipy import sparse
import scipy.stats as ss
from scipy.sparse import csr_matrix, hstack, vstack

import matplotlib.pyplot as plt, matplotlib.gridspec as gridspec 
import seaborn as sns
from wordcloud import WordCloud ,STOPWORDS
from PIL import Image

import string, re, nltk, collections
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer   

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split


In [54]:
# import the dataset

train = pd.read_csv("V1.4_Training.csv", encoding = 'latin-1')
dev = pd.read_csv("SubtaskA_Trial_Test_Labeled.csv", encoding = 'latin-1')
test = pd.read_csv("SubtaskA_EvaluationData.csv", encoding = 'latin-1')

<div><h3 id = "eda"> Exploratory Data Analysis </h3></div>

In [55]:

train = train.rename(columns={'663_3': 'id','"Please enable removing language code from the Dev Center "language history" For example if you ever selected "ru" and "ru-ru" laguages and you published this xap to the Store then it causes Tile localization to show the en-us(default) tile localization which is bad."': 'sentence','1': 'label'})
train.head(10)

,id,sentence,label
0,663_4,"""Note: in your .csproj file, there is a Suppor...",0
1,664_1,"""Wich means the new version not fully replaced...",0
2,664_2,"""Some of my users will still receive the old x...",0
3,664_3,"""The store randomly gives the old xap or the n...",0
4,664_4,"""My app has a WP7 version and a WP8 version XA...",0
5,664_5,"""The wp7 xap works only on WP7 and the wp8 xap...",0
6,665_1,"""Sometimes the Store gives the wrong wp7 xap v...",0
7,665_2,"""It should be an option to remove the ""ru"" lan...",1
8,665_3,"""Currently if you ever mistakenly selected a ""...",0
9,665_5,"""): the store will randomly deliver the old/wr...",0


In [56]:
print("Training data...")
train.info()

Training data...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8499 entries, 0 to 8498
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        8499 non-null   object
 1   sentence  8499 non-null   object
 2   label     8499 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 199.3+ KB


In [57]:
# class-imbalance in training data

suggestion_count = (train['label'].values == 1).astype(int).sum()
non_suggestion_count = (train['label'].values == 0).astype(int).sum()
print("Total sentences : " + str(train.shape[0]))
print("Total suggestions : " + str(suggestion_count))
print("Total non_suggestions : " + str(non_suggestion_count))

Total sentences : 8499
Total suggestions : 2084
Total non_suggestions : 6415


In [58]:
# oversampling to balance the training data

suggestions = train[train['label'].values == 1]

while suggestion_count < non_suggestion_count:
    random_suggestion = suggestions.sample()
    train = train.append(random_suggestion, ignore_index = True)
    suggestion_count = suggestion_count + 1

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12830 entries, 0 to 12829
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        12830 non-null  object
 1   sentence  12830 non-null  object
 2   label     12830 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 300.8+ KB


In [59]:
# exploring the development/validation data

print("Development Set...")
dev.info()

Development Set...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 592 entries, 0 to 591
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        592 non-null    object
 1   sentence  592 non-null    object
 2   label     592 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 14.0+ KB


In [60]:
# class-imbalance in development data

suggestion_count = (dev['label'].values == 1).astype(int).sum()
non_suggestion_count = (dev['label'].values == 0).astype(int).sum()
print("Total sentences : " + str(dev.shape[0]))
print("Total suggestions : " + str(suggestion_count))
print("Total non_suggestions : " + str(non_suggestion_count))

Total sentences : 592
Total suggestions : 296
Total non_suggestions : 296


In [61]:
# Aphost lookup dict

APPO = {
    "aren't" : "are not",
    "can't" : "cannot",
    "couldn't" : "could not",
    "didn't" : "did not",
    "doesn't" : "does not",
    "don't" : "do not",
    "hadn't" : "had not",
    "hasn't" : "has not",
    "haven't" : "have not",
    "he'd" : "he would",
    "he'll" : "he will",
    "he's" : "he is",
    "i'd" : "I would",
    "i'd" : "I had",
    "i'll" : "I will",
    "i'm" : "I am",
    "isn't" : "is not",
    "it's" : "it is",
    "it'll":"it will",
    "i've" : "I have",
    "let's" : "let us",
    "mightn't" : "might not",
    "mustn't" : "must not",
    "shan't" : "shall not",
    "she'd" : "she would",
    "she'll" : "she will",
    "she's" : "she is",
    "shouldn't" : "should not",
    "that's" : "that is",
    "there's" : "there is",
    "they'd" : "they would",
    "they'll" : "they will",
    "they're" : "they are",
    "they've" : "they have",
    "we'd" : "we would",
    "we're" : "we are",
    "weren't" : "were not",
    "we've" : "we have",
    "what'll" : "what will",
    "what're" : "what are",
    "what's" : "what is",
    "what've" : "what have",
    "where's" : "where is",
    "who'd" : "who would",
    "who'll" : "who will",
    "who're" : "who are",
    "who's" : "who is",
    "who've" : "who have",
    "won't" : "will not",
    "wouldn't" : "would not",
    "you'd" : "you would",
    "you'll" : "you will",
    "you're" : "you are",
    "you've" : "you have",
    "'re": " are",
    "wasn't": "was not",
    "we'll":" will",
    "didn't": "did not",
    "tryin'":"trying"
}

In [1]:
def clean(sentence):
    sentence = sentence.lower()
    sentence = re.sub('<.*>', '', sentence)
    sentence = re.sub("\\n", "", sentence)
    sentence = re.sub("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", "", sentence)
    sentence = re.sub("\[\[.*\]", "", sentence)
    sentence = re.sub("[" + re.sub("\.","",string.punctuation) + "]", "", sentence)
    
    words = tokenizer.tokenize(sentence)
    
    words = [APPO[word] if word in APPO else word for word in words]
    words = [ps.stem(word) for word in words]
    words = [lem.lemmatize(word, "v") for word in words]
    words = [w for w in words if not w in eng_stopwords]
    
    clean_sent = " ".join(words)
    
    return(clean_sent)

In [62]:
# obtaining separate clean corpora for suggestion and non-suggestion classes

suggestion_corpus = train[train['label'].values == 1].sentence
suggestion_corpus = suggestion_corpus.append(dev[dev['label'].values == 1].sentence)
#clean_suggestion_corpus = ""
#for sentence in suggestion_corpus:
   # clean_suggestion_corpus += clean(sentence)

non_suggestion_corpus = train[train['label'].values == 0].sentence
non_suggestion_corpus = non_suggestion_corpus.append(dev[dev['label'].values == 0].sentence)
#clean_non_suggestion_corpus = ""
#for sentence in non_suggestion_corpus:
    #clean_non_suggestion_corpus += clean(sentence)

In [66]:
# top 20 bigrams in suggestion sentences

suggestion_bigrams = ngrams(suggestion_corpus.split(), 2)
suggestion_bigrams_freq = collections.Counter(suggestion_bigrams)
suggestion_bigrams_freq.most_common(20)

AttributeError: 'Series' object has no attribute 'split'

In [65]:
# top 20 bigrams in non-suggestion sentences

non_suggestion_bigrams = ngrams(non_suggestion_corpus.split(), 2)
non_suggestion_bigrams_freq = collections.Counter(non_suggestion_bigrams)
non_suggestion_bigrams_freq.most_common(20)

[]

In [16]:
del(suggestions)
del(subset)
del(content)
del(stopword)
del(wc)
del(suggestion_corpus)
del(clean_suggestion_corpus)
del(non_suggestion_corpus)
del(clean_non_suggestion_corpus)
gc.collect()

0


<h3 id="fe"> Feature Engineering </h3>


In [19]:
# corpus containing all the sentences in train, development and test data

corpus = (pd.concat([train.iloc[:, 0:2], dev.iloc[:, 0:2], test.iloc[:, 0:2]])).sentence
clean_corpus = corpus.apply(lambda x : clean(x))

In [21]:
# tf-idf vectors with bigram and trigram features

btgram_tfv = TfidfVectorizer(strip_accents = 'unicode', analyzer = 'word', ngram_range = (2,3),
            sublinear_tf = 1, stop_words = 'english')
btgram_tfv.fit(clean_corpus)

train_btgrams = btgram_tfv.transform(clean_corpus.iloc[:train.shape[0]])
dev_btgrams = btgram_tfv.transform(clean_corpus.iloc[train.shape[0]:train.shape[0]+dev.shape[0]])
test_btgrams = btgram_tfv.transform(clean_corpus.iloc[train.shape[0]+dev.shape[0]:])


    <h3>Evaluation Metrics</h3>


In [23]:
# evaluation functions 

def lgb_f1_score(preds, train_data):
    y_train = train_data.get_label()
    preds = (preds >= 0.5).astype(int)
    return 'f1_score', f1_score(y_train, preds), True



In [24]:
# dataframes for blending

train_labels = pd.DataFrame()
dev_labels = pd.DataFrame()

<div>
    <h3 Statistical Models </h3>
    <p> Logistic Regression and Support Vector Machine (SVM).</p>
</div>

In [25]:

x_train = hstack((train_unigrams, train_btgrams, train_charngrams)).tocsr()
y_train = train['label'].values
x_dev = hstack((dev_unigrams, dev_btgrams, dev_charngrams)).tocsr()
y_dev = dev['label'].values
x_test = hstack((test_unigrams, test_btgrams, test_charngrams)).tocsr()

In [26]:
# logistic regression classifier

clf = LogisticRegression(C = 0.1, solver = 'liblinear')
clf.fit(x_train, y_train)

lr_dev_pred = clf.predict_proba(x_dev)[:, 1]
lr_test_pred = clf.predict_proba(x_test)[:, 1]

train_labels['lr'] = (clf.predict_proba(x_train)[:, 1] >= 0.5).astype(int)
dev_labels['lr'] = (clf.predict_proba(x_dev)[:, 1] >= 0.5).astype(int)

y_pred = (lr_dev_pred >= 0.5).astype(int)
lr_precision = precision_score(y_dev, y_pred)
lr_recall = recall_score(y_dev, y_pred)
lr_f1 = f1_score(y_dev, y_pred)

print("Logistic Regression...")
print("Precision score : " + str(lr_precision))
print("Recall score : " + str(lr_recall))
print("F1 score : " + str(lr_f1))

Logistic Regression...
Precision score : 0.7489878542510121
Recall score : 0.625
F1 score : 0.6813996316758747


In [27]:
# SVM classifier

# reducing the number of features using Singular Value Decomposition
svd = TruncatedSVD(n_components = 15)
svd.fit(vstack((x_train, x_dev, x_test)).tocsr())
x_train_svd = svd.transform(x_train)
x_dev_svd = svd.transform(x_dev)
x_test_svd = svd.transform(x_test)

scaler = StandardScaler()
scaler.fit(np.concatenate((x_train_svd, x_dev_svd, x_test_svd)))
x_train_svd = scaler.transform(x_train_svd)
x_dev_svd = scaler.transform(x_dev_svd)
x_test_svd = scaler.transform(x_test_svd)

clf = SVC(C = 0.1, probability = True)
clf.fit(x_train_svd, y_train)

svm_dev_pred = clf.predict_proba(x_dev_svd)[:, 1]
svm_test_pred = clf.predict_proba(x_test_svd)[:, 1]

train_labels['svm'] = (clf.predict_proba(x_train_svd)[:, 1] >= 0.5).astype(int)
dev_labels['svm'] = (clf.predict_proba(x_dev_svd)[:, 1] >= 0.5).astype(int)

y_pred = (svm_dev_pred >= 0.5).astype(int)
svm_precision = precision_score(y_dev, y_pred)
svm_recall = recall_score(y_dev, y_pred)
svm_f1 = f1_score(y_dev, y_pred)

print("SVM Classifier...")
print("Precision score : " + str(svm_precision))
print("Recall score : " + str(svm_recall))
print("F1 score : " + str(svm_f1))

SVM Classifier...
Precision score : 0.697508896797153
Recall score : 0.6621621621621622
F1 score : 0.6793760831889081
